In [1]:
import numpy as np
import glob, os
from keras import layers as ly
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import keras.backend.tensorflow_backend as K
from keras.models import load_model
from keras.optimizers import Adam
from keras.layers import Input, Lambda
from keras.models import Model

Using TensorFlow backend.


In [2]:
#dir = './sequence/*'
dir = './latest_sequence/bfs/*'

In [3]:
# file read
all_data = []
sequence_length = []
for file in sorted(glob.glob(dir)):
    datasets = []
    for f in open(file, 'r'):
        f = f.replace(']', '').replace('[', '').replace('\n','')
        (u, v, w) = f.split(',')
        datasets.append([int(u), int(v), float(w)])
    sequence_length.append(len(datasets))
    all_data.append(datasets)
#all_data = np.array(all_data)
all_data = np.array([np.array(arr) for arr in all_data])

In [4]:
x_train, x_test = train_test_split(all_data, test_size = 0.2)
x_test, x_val = train_test_split(x_test, test_size = 0.66)

In [5]:
max_sequence_length = max(sequence_length)
n_features = 3
batch_size = 32
epochs = 100
steps_per_epoch = len(x_train)

In [8]:


def repeat_vector(args):
    layer_to_repeat = args[0]
    sequence_layer = args[1]
    return RepeatVector(K.shape(sequence_layer)[1])(layer_to_repeat)
    

inputs = Input(shape=(None, 3))
encoded = LSTM(128, return_sequences=True)(inputs)  #activation 안적으면 tanh
encoded = LSTM(64)(encoded)

decoded = Lambda(repeat_vector, output_shape=(None, 64)) ([encoded, inputs]) # inputs의 shape[1] 만큼 encoded 를 반복 생성

decoded = LSTM(64, return_sequences=True)(decoded)
decoded = LSTM(128, return_sequences=True)(decoded)
decoded = TimeDistributed(Dense(3))(decoded)
encoder = Model(inputs, encoded)

lstm_autoencoder = Model(inputs, decoded)
lstm_autoencoder.compile(loss='mean_squared_error', optimizer=Adam(lr=0.0006))
lstm_autoencoder_250 = lstm_autoencoder

In [9]:
def train_generator(x_train):
    idx = 0
    while True:
        yield np.array([x_train[idx]]), np.array([x_train[idx]])
        idx +=1
        if idx >= len(x_train):
            idx = 0

In [ ]:
lstm_autoencoder.fit_generator(train_generator(x_train), epochs=200, steps_per_epoch=steps_per_epoch, verbose=1)#, validation_data=val_generator(x_val))

'''
- epochs 467 - 0.1900
- epochs 100 해서 확인해보면 loss 가 너무 들쑥날쑥한 걸 확인할 수 있음 1.22
- epochs 500 0.2346

- 데이터 셋 변경 epochs 500, loss - 0.0572, lr-0.001
  lr-0.0006 0.0547
'''

Epoch 1/200
2688/2688 [==============================] - 24s 9ms/step - loss: 37.5512
Epoch 2/200
2688/2688 [==============================] - 22s 8ms/step - loss: 36.3563
Epoch 3/200
2688/2688 [==============================] - 22s 8ms/step - loss: 21.9553
Epoch 4/200
2688/2688 [==============================] - 24s 9ms/step - loss: 13.4853
Epoch 5/200
2688/2688 [==============================] - 25s 9ms/step - loss: 11.8289
Epoch 6/200
2688/2688 [==============================] - 24s 9ms/step - loss: 10.5707
Epoch 7/200
2688/2688 [==============================] - 27s 10ms/step - loss: 9.6544
Epoch 8/200
2688/2688 [==============================] - 32s 12ms/step - loss: 8.9367
Epoch 9/200
2688/2688 [==============================] - 32s 12ms/step - loss: 8.1741
Epoch 10/200
2688/2688 [==============================] - 31s 12ms/step - loss: 7.1135
Epoch 11/200
2688/2688 [==============================] - 31s 12ms/step - loss: 6.3514
Epoch 12/200
2688/2688 [============================

2688/2688 [==============================] - 31s 11ms/step - loss: 0.5175
Epoch 96/200
2688/2688 [==============================] - 31s 11ms/step - loss: 0.5175
Epoch 97/200
2688/2688 [==============================] - 31s 12ms/step - loss: 0.4889
Epoch 98/200
2688/2688 [==============================] - 31s 11ms/step - loss: 0.3013
Epoch 99/200
2688/2688 [==============================] - 31s 12ms/step - loss: 0.2131
Epoch 100/200
2688/2688 [==============================] - 31s 11ms/step - loss: 0.2731
Epoch 101/200
2688/2688 [==============================] - 31s 11ms/step - loss: 0.4394
Epoch 102/200
2688/2688 [==============================] - 31s 11ms/step - loss: 0.3274
Epoch 103/200
2688/2688 [==============================] - 31s 12ms/step - loss: 0.2613
Epoch 104/200
2688/2688 [==============================] - 31s 11ms/step - loss: 0.2239
Epoch 105/200
2688/2688 [==============================] - 31s 11ms/step - loss: 0.2175
Epoch 106/200
2688/2688 [=========================

In [ ]:
lstm_autoencoder_250.fit_generator(train_generator(x_train), epochs=500, steps_per_epoch=steps_per_epoch, verbose=1)#, validation_data=val_generator(x_val))

In [ ]:
# model save
lstm_autoencoder.save('model/lstm_autoencoder200_bfs.h5')

In [ ]:
loaded_model = load_model('model/lstm_autoencoder_bfs.h5')

In [ ]:
loaded_model.summary()

In [ ]:
mean= 0
for xt in x_test:
    xt = xt.reshape(1, xt.shape[0], xt.shape[1])
    out = loaded_model.predict(xt)
    mean += ((xt-out)**2).mean(axis=None)
print(mean/len(x_test))

In [19]:
encoder = Model(loaded_model.input, loaded_model.layers[3].output)

In [40]:
decoder_layer = loaded_model.layers[-1]
decoder = Model(loaded_model.layers[2].output, 
                        loaded_model.layers[6].output)

AttributeError: Layer time_distributed_3 has multiple inbound nodes, hence the notion of "layer output" is ill-defined. Use `get_output_at(node_index)` instead.

In [76]:
test = x_test[0].reshape(1, 9, 3)
latent_vector = []
for x in x_test:
    x = x.reshape(1, x.shape[0], x.shape[1])
    latent_vector.append(encoder.predict(x)[0])

In [161]:
latent_vector = np.array(latent_vector)

[-0.0,
 -0.00020211579,
 0.0,
 0.0,
 0.74070954,
 -0.029288653,
 -0.74629986,
 0.0,
 0.12747759,
 -0.0014260766,
 -0.0,
 0.10494814,
 0.2187337,
 -0.18827741,
 0.040405355,
 -0.5645764,
 0.0022893408,
 0.0,
 0.0,
 -0.2676836,
 -0.7395759,
 0.2642008,
 -0.41065842,
 0.19330677,
 -0.3758831,
 -0.21791713,
 0.055199105,
 -0.25927162,
 0.23109113,
 0.8425839,
 -0.03407281,
 -0.8132615,
 0.0,
 0.49131706,
 0.16162786,
 0.35315764,
 -0.0,
 -0.7594612,
 -0.6364968,
 0.0,
 0.009271684,
 0.0,
 0.30094486,
 0.0,
 -0.0027280087,
 -0.25117594,
 0.0,
 0.19940257,
 0.32026434,
 0.5843708,
 0.032265846,
 -0.0,
 -0.0140618635,
 -0.0,
 0.0,
 0.0,
 -0.0,
 0.254268,
 -0.0,
 0.0,
 -0.0,
 0.19169566,
 0.0,
 0.7144473]

In [144]:
def printMatrix(matrix):
    length = int(max(max(matrix[:, 0]), max(matrix[:, 1])))
    adj = [[0 for i in range(length)] for i in range(length)]
    
    for m in matrix:
        (i, j, w) = m
        i = int(i) -1
        j = int(j) -1
        adj[i][j] = adj[j][i] = w
    for i in range(length):
        for j in range(length):
            print(str(adj[i][j]), end=',')
        print()
    print("===============")

In [213]:
# sequence to matrix
for index, x in enumerate(x_test):
    print("#", str(index))
    printMatrix(x)
    

# 0
0,10.16,7.92,9.56,0,0,0,0,0,
10.16,0,0,0,0,0,0,0,0,
7.92,0,0,0,8.17,18.05,0,0,0,
9.56,0,0,0,0,0,20.05,0,0,
0,0,8.17,0,0,0,0,0,0,
0,0,18.05,0,0,0,0,32.09,0,
0,0,0,20.05,0,0,0,0,20.27,
0,0,0,0,0,32.09,0,0,9.01,
0,0,0,0,0,0,20.27,9.01,0,
# 1
0,8.79,11.79,0,0,0,
8.79,0,0,10.06,6.86,0,
11.79,0,0,0,0,7.86,
0,10.06,0,0,0,0,
0,6.86,0,0,0,0,
0,0,7.86,0,0,0,
# 2
0,3.3,6.3,0,0,
3.3,0,0,12.72,0,
6.3,0,0,0,23.87,
0,12.72,0,0,41.95,
0,0,23.87,41.95,0,
# 3
0,5.7,7.22,0,0,
5.7,0,0,12.22,0,
7.22,0,0,0,23.54,
0,12.22,0,0,42.39,
0,0,23.54,42.39,0,
# 4
0,12.52,17.52,28.77,0,0,0,0,0,
12.52,0,0,0,0,0,0,0,0,
17.52,0,0,0,2.53,4.53,0,0,0,
28.77,0,0,0,0,0,10.86,0,0,
0,0,2.53,0,0,0,0,0,0,
0,0,4.53,0,0,0,0,9.01,0,
0,0,0,10.86,0,0,0,0,10.01,
0,0,0,0,0,9.01,0,0,20.01,
0,0,0,0,0,0,10.01,20.01,0,
# 5
0,4.27,9.29,0,0,
4.27,0,0,12.29,0,
9.29,0,0,0,17.29,
0,12.29,0,0,3.95,
0,0,17.29,3.95,0,
# 6
0,10.16,7.92,9.56,0,0,0,0,0,
10.16,0,0,0,0,0,0,0,0,
7.92,0,0,0,8.17,18.05,0,0,0,
9.56,0,0,0,0,0,20.05,0,0,
0,0,8.17,0,0,0,0

In [171]:
standard = latent_vector[0]

In [223]:
import copy
dist = []
left = 100000
right = -10000
for index, v in enumerate(latent_vector[1:]):
    d = round(euclidean_distance(standard, v), 8)
    dist.append(d)
    if d > right:
        right= d
    if left > d:
        left = d
dist1 = copy.deepcopy(dist)
dist1.sort()
#print(dist, dist1)
for index, d in enumerate(dist1):
    print( (index+1, dist.index(d)+1), end='  ')

(1, 2)  (2, 38)  (3, 53)  (4, 5)  (5, 15)  (6, 44)  (7, 48)  (8, 21)  (9, 12)  (10, 43)  (11, 9)  (12, 18)  (13, 31)  (14, 59)  (15, 58)  (16, 4)  (17, 41)  (18, 27)  (19, 29)  (20, 33)  (21, 6)  (22, 3)  (23, 51)  (24, 35)  (25, 54)  (26, 55)  (27, 26)  (28, 62)  (29, 14)  (30, 46)  (31, 11)  (32, 45)  (33, 7)  (34, 50)  (35, 57)  (36, 8)  (37, 63)  (38, 60)  (39, 24)  (40, 52)  (41, 20)  (42, 13)  (43, 37)  (44, 23)  (45, 10)  (46, 1)  (47, 61)  (48, 42)  (49, 56)  (50, 34)  (51, 16)  (52, 32)  (53, 40)  (54, 39)  (55, 19)  (56, 17)  (57, 22)  (58, 49)  (59, 30)  (60, 36)  (61, 47)  (62, 28)  (63, 25)  

In [168]:
def euclidean_distance(a, b):
    return np.sqrt(np.sum(a-b) ** 2)

In [221]:
len(x_test)

64